In [2]:
!pip install pyreadstat==1.2.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 22.1 MB/s eta 0:00:00


In [3]:
import pyreadstat

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Используем данные 2020 года, ибо в более ранних нет вопроса про дистанционную работу

In [12]:
fpath = 'drive/MyDrive/коуж/IND_OSN_2020.sav' # Источник данных: https://gks.ru .

In [13]:
df, meta = pyreadstat.read_file_multiprocessing(pyreadstat.read_sav, fpath, num_processes=16, encoding='cp1251') 
df.head()

,H00_02,H00_04,H00_06,H01_00,H00_07,H01_01,H01_02,H01_03,H01_04,H01_05,...,PM_DX_SRED_MES,MALOIM,KVZV,DLIT_OTP,ROD_NER_INV,ROD_BEZR,ROD_UCH_OCH,ROD_ALK,NAL_ROD,SIROT
0,01,1.0,1.0,1.0,1.0,2.0,49.0,0.0,3.0,NaN,...,11190.0,0.0,409.261312,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,01,1.0,2.0,1.0,1.0,1.0,40.0,0.0,5.0,NaN,...,11190.0,0.0,1167.354856,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,01,1.0,3.0,1.0,1.0,1.0,70.0,0.0,4.0,NaN,...,8960.0,0.0,727.831661,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,01,1.0,4.0,1.0,1.0,1.0,66.0,0.0,4.0,NaN,...,8960.0,0.0,727.831661,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,01,1.0,5.0,1.0,1.0,2.0,69.0,0.0,4.0,NaN,...,8960.0,0.0,1100.360119,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Что такое частота - читаем туть (да, это сайт с оформлением из нулевых и смешной рекламой, но зато всё понятно): http://mathprofi.ru/matematicheskaya_statistika.html

In [19]:
columns_to_count_frequencies = [
    'R_2_0', 'I02_01_01', 'MNG_SEM', 'R_2_5', 'NP_SEM', 'R_2_6', 'MOL_SEM_D_1', 'R_3_1', 'CH_UKR', 'PM_DX_SRED_MES', 'I02_02', 'I02_03_15', 'I02_03_14', 'I02_03_13', 'I02_03_01', 'I09_09', 'I10_01',
    'I10_41', 'I10_42_03', 'I10_42_02', 'I10_42_01', 'I10_39', 'I10_40', 'I09_03_06_01', 'I09_03_06_02', 'I09_06', 'I09_08', 'I09_03_07_01', 'I09_03_07_02', 'I09_03_05_01', 'I09_03_05_02', 
    'I09_081_01', 'I09_03_04_01', 'I09_03_04_02', 'I09_03_03_01', 'I09_03_03_02', 'I09_051', 'I09_05_08', 'I09_04', 'I02_01_01', 'I02_02', 'I02_03_01', 'I02_03_13', 'I02_03_14', 'I02_03_15',
    'I04_04_01', 'I04_04_03', 'I05_01', 'I05_02', 'I05_04', 'I05_06', 'I05_10', 'I05_11', 'I05_14', 'I05_15', 'I05_151', 'I05_17_01', 'I05_20_01', 'I05_20_07', 'I05_20_08', 'I05_29', 'I05_37',
    'I05_38', 'I05_41', 'I06_06_01_02', 'I06_06_02_02', "I06_06_03_02", "I06_06_04_02", "I07_01", "I07_04", "I07_05", "I07_17_01", "I07_17_02", "I07_18", "I08_04", "I08_05_05", "I08_05_09",
    "I08_07", "I09_01_02", "I09_01_03", "I09_01_04", "I09_012_07", "I09_03_01_01", "I09_03_01_02", "I09_03_02_01", "I09_03_02_02", "D03_45"
    ]

In [35]:
# PM_DX_SRED_MESВПМ на члена ДХ, месяц (рублей) - нет в словаре meta.variable_to_label

документация pandas - есть наглядный пример разницы между частотами и частостями (у них они называются relative frequencies)
https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html


In [34]:
for column in columns_to_count_frequencies:
    try:
        print(meta.column_names_to_labels[column])
        frequencies = df[column].value_counts(dropna=False)
        # value_counts возвращает Series, где value это индекс, а значения - частоты или частости
        for name, freq in zip(frequencies.index, frequencies):
            print(f"Частота {meta.value_labels[meta.variable_to_label[column]].get(name)}: {freq}")
        frequencies = df[column].value_counts(normalize=True, dropna=False)
        for name, freq in zip(frequencies.index, frequencies):
            print(f"Относительная частота (частость) {meta.value_labels[meta.variable_to_label[column]].get(name)}: {freq}")
    except KeyError:
        print(f"{column} отсутствует в meta")
    print('-------------------------------------------------------------------')


Состав домохозяйства
Частота из 2 человек: 41586
Частота из 3 человек: 29232
Частота из 4 человек: 23388
Частота из 1 человека: 20830
Частота 5 человек и более: 12420
Относительная частота (частость) из 2 человек: 0.3262773035400452
Относительная частота (частость) из 3 человек: 0.22934973637961337
Относительная частота (частость) из 4 человек: 0.18349861913130805
Относительная частота (частость) из 1 человека: 0.16342894802912378
Относительная частота (частость) 5 человек и более: 0.09744539291990961
-------------------------------------------------------------------
В своем населенном пункте респондент проживает с рождения
Частота None: 76215
Частота С рождения: 51241
Относительная частота (частость) None: 0.597971064524228
Относительная частота (частость) С рождения: 0.402028935475772
-------------------------------------------------------------------
Признак многодетной семьи
Частота None: 119733
Частота Многодетная семья: 7723
Относительная частота (частость) None: 0.9394065402962